<a href="https://colab.research.google.com/github/dominicwhite/kaggle-club/blob/master/python/linear_regression_and_friends.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear regression and friends

## The `diabetes` dataset

We will be using sklearn's built-in diabetes dataset.

In [27]:
from sklearn import datasets

diabetes = datasets.load_diabetes()

In [28]:
print(diabetes.DESCR)
print(diabetes.feature_names)

.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

  :Number of Instances: 442

  :Number of Attributes: First 10 columns are numeric predictive values

  :Target: Column 11 is a quantitative measure of disease progression one year after baseline

  :Attribute Information:
      - Age
      - Sex
      - Body mass index
      - Average blood pressure
      - S1
      - S2
      - S3
      - S4
      - S5
      - S6

Note: Each of these 10 feature variables have been mean centered and scaled by the standard deviation times `n_samples` (i.e. the sum of squares of each column totals 1).

Source URL:
https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html

For more information see:
Bra

In [29]:
diabetes_X = diabetes.data
diabetes_y = diabetes.target

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(diabetes_X, diabetes_y, test_size = 0.2)

## Basic Linear regression in `scikit-learn`

![alt text](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3a/Linear_regression.svg/438px-Linear_regression.svg.png)

For linear regression, we can use the `LinearRegression` class.


In [30]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()

lin_reg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

We can now look at the linear model we just created.

The coefficients are stored in the `coef_` attribute.

> **Pro-tip**: any attributed created by the `fit(...)` method will end with an underscore.

In [31]:
print(lin_reg.coef_)

[   8.99632085 -250.8480026   563.13634469  294.92140304 -904.9661982
  470.10412496  162.68727107  268.50280092  809.68561994   36.01795276]


Let's make this a bit more readable by associating each coefficient with its variable name.

In [32]:
import pandas as pd

pd.DataFrame(lin_reg.coef_, index = diabetes.feature_names)

,0
age,8.996321
sex,-250.848003
bmi,563.136345
bp,294.921403
s1,-904.966198
s2,470.104125
s3,162.687271
s4,268.502801
s5,809.685620
s6,36.017953


What about the intercept?

In [33]:
print('Intercept: \n', lin_reg.intercept_)

Intercept: 
 150.77434173919147


In [34]:
lin_reg_y_pred = lin_reg.predict(X_test)

We can calculate some error metrics

In [35]:
from sklearn.metrics import mean_squared_error, r2_score

print('Mean squared error: %.2f'
      % mean_squared_error(y_test, lin_reg_y_pred))
# The R-squared value (aka coefficient of determination): 1 is perfect prediction
print('R-squared: %.2f'
      % r2_score(y_test, lin_reg_y_pred))


Mean squared error: 2994.83
R-squared: 0.46


## How does linear regression work?

We find the line that minimizes the sum of the squared residuals.

This is also known as the mean squared error (MSE).

It turns out that the MSE is always a parabola w.r.t. the coefficients of the model:

![alt text](https://www.onlinemath4all.com/images/minimumvalueofparabola.png)

For linear regression we can find an exact solution.



*   Differentiate the cost function equation (MSE)
*   Set the differential == 0.
*   Solve!



## Polynomial regression aka. how to get lots of features

A standard linear regression might look like this:

$y = m_1 x_1 + m_2 x_2 + c$

But this assumes that y and each x are linearly related.

Maybe they aren't in which case we can transform the variables, e.g. by taking their logarithm, or by calculating polynomial variations:

$y = m_1 x_1 + m_2 x_2 + m_3 x_1^2 + m_4 x_2^2 + c$

In [36]:
from sklearn.preprocessing import PolynomialFeatures

poly_features = PolynomialFeatures(degree=2, include_bias = False)
X_train_poly = poly_features.fit_transform(X_train)

lin_reg_poly = LinearRegression()
lin_reg_poly.fit(X_train_poly, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [40]:
X_train_poly

array([[-1.64121703e-02, -4.46416365e-02, -1.05172024e-02, ...,
         4.57689584e-04,  7.31975227e-04,  1.17063562e-03],
       [ 1.26481373e-02, -4.46416365e-02, -2.56065715e-02, ...,
         5.20251396e-03, -8.18558656e-04,  1.28791250e-04],
       [-1.88201653e-03, -4.46416365e-02,  5.41515220e-02, ...,
         7.13945272e-03,  4.10880156e-03,  2.36464207e-03],
       ...,
       [ 3.44433680e-02,  5.06801187e-02, -9.43939036e-03, ...,
         1.21178289e-04, -2.39847060e-04,  4.74727057e-04],
       [-5.27375548e-02,  5.06801187e-02, -1.15950145e-02, ...,
         9.34310152e-04, -1.59551086e-04,  2.72463581e-05],
       [-1.88201653e-03, -4.46416365e-02, -2.66843835e-02, ...,
         1.78486875e-02,  2.62292701e-03,  3.85448292e-04]])

## Regularized Linear Models

If we have a lot of features in our training data, we are likely to overfit.

However, we don't want to arbitrarily exclude features - after all they might include useful information.

One solution is **regularization**: we penalize features in our cost function as well as the fit.

For example, in standard linear regression:

cost_function = MSE

In regularized linear regression:

cost_function = MSE + regularization

Alternatively we can find an approximation of the minimum using gradient descent:

![alt text](https://images.deepai.org/glossary-terms/dd6cdd6fcfea4af1a1075aac0b5aa110/sgd.png)

This can be a lot quicker when you have large amounts of data (lots of rows), or a lot of features in X (lots of columns).

However, you have to decide on a learning rate (don't want this to be too big or too small).

### Ridge regression

In ridge regression, the regularization term is the sum of the coefficients squared.

So, our cost function is:

$cost function = MSE + \alpha \sum{(coefs)^2}$

where 
* $\alpha$ is a constant set by **you** that determines how strong this regularization penalty is.

There's a great video on Youtube that introduces Ridge regression very simply and shows how it's predictions compare to standard regression (~20 mins).

> *Note: the narrator recommends a bunch of other videos to watch first, but you can ignore most of these. The Bias-Variance video might be useful if you have never heard of those concepts, but you can also just think of "Bias" as underfitting, and "Variance" as overfitting.*

<a href="http://www.youtube.com/watch?feature=player_embedded&v=Q81RR3yKn30
" target="_blank"><img src="http://img.youtube.com/vi/Q81RR3yKn30/0.jpg" 
alt="Elastic Net" width="480" height="360" border="1" /></a>

In Scikit-Learn, we can use the `Ridge` class to do Ridge regression just like we did standard linear regression (more details on Ridge regression with sklearn [in the official guide](https://scikit-learn.org/stable/modules/linear_model.html#ridge-regression))

In [ ]:
from sklearn.linear_model import Ridge

ridge_reg = Ridge(alpha = 1) # note that we have to set alpha

ridge_reg.fit(X_train, y_train)

#### Exercises

1. Repeat the steps we carried out for the linear regression above to look at the models fit and accuracy.

2. What happens if you set alpha to 0?

### Lasso regression

Like ridge regression, but our regularization term is sum of the absolute values of the coefficients (rather than their squares).

$cost function = MSE + \alpha \sum{|coefs|}$

The same Youtube channel as before has another (even shorter) video on Lasso regression:

<a href="http://www.youtube.com/watch?feature=player_embedded&v=NGf0voTMlcs
" target="_blank"><img src="http://img.youtube.com/vi/NGf0voTMlcs/0.jpg" 
alt="Elastic Net" width="480" height="360" border="1" /></a>

In Scikit-Learn, we can use the `Lasso` class to do Lasso regression just like we did standard or Ridge regression (more details on Lasso regression with sklearn [in the official guide](https://scikit-learn.org/stable/modules/linear_model.html#lasso))

In [41]:
from sklearn.linear_model import Lasso

lasso_reg = Lasso(alpha = 1) # note that we have to set alpha

lasso_reg.fit(X_train, y_train)

Lasso(alpha=1, copy_X=True, fit_intercept=True, max_iter=1000, normalize=False,
      positive=False, precompute=False, random_state=None, selection='cyclic',
      tol=0.0001, warm_start=False)

#### Exercise

1. Repeat the steps we carried out for the linear regression above to look at the models fit and accuracy.

### More exercises

1. We can combine Ridge and Lasso regression into the same regression model. This is called **ElasticNet** and we can use a sklearn class of the same name: `ElasticNet` (more details in the [documentation](https://scikit-learn.org/stable/modules/linear_model.html#ridge-regression)). Try using Elastic Net to create an a different regression model.

  You can learn more about Elastic Net in this Youtube video:

  <a href="http://www.youtube.com/watch?feature=player_embedded&v=1dKRdX9bfIo
" target="_blank"><img src="http://img.youtube.com/vi/1dKRdX9bfIo/0.jpg" 
alt="Elastic Net" width="480" height="360" border="1" /></a>

2. We have to set alpha (the model can't figure it out for us). How do we decide the best value? **Cross-validation!**

  i. One way of getting the optimal value of alpha with cross validation is to manually search through values of alpha using the `GridSearchCV` class. Link to documentation 

  ii. Alternatively there are versions of each regularized linear regression which will do cross validation much more simply:
    * [LassoCV](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html)
    * [RidgeCV](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html)
    * [ElasticNetCV](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNetCV.html)

    For example, you can use code like this:

    ```python
    from sklearn.linear_model import ElasticNetCV

    # Values of alpha to try:
    alphas = [0.1, 0.5, 1, 2, 4, 8]

    elastic_cv = ElasticNetCV(alphas=alphas, cv=5)
    elastic_cv.fit(X_train, y_train)
    
    # Then we use elastic_cv like any of the previous models:
    print(elastic_cv.alpha_)
    print(elastic_cv.intercept_)
    ```